In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Helpful
import tqdm

# Visualization
import matplotlib.pyplot as plt

# My Methods
from src.utils.CRPS import *
from src.utils.data_split import *
from src.models.EMOS import *
import data.raw.load_data_raw as ldr
import data.processed.load_data_processed as ldp
import data.processed.load_data_processed_denormed as ldpd
from src.models.CRPS_baseline.CRPS_load import *

2023-05-28 14:42:00.274943: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/opt/anaconda3/lib/
2023-05-28 14:42:00.274962: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


<IPython.core.display.Javascript object>

### 1. Load Raw Data

In [13]:
# Load train data for variables t2m and ws10
train_t2m_ws10_denormed = ldpd.load_data_t2m_ws10_train_proc_denorm()
train_t2m_denormed = train_t2m_ws10_denormed[0]
train_ws10_denormed = train_t2m_ws10_denormed[1]

# Load test data for variables t2m and ws10
test_t2m_ws10_denormed = ldpd.load_data_t2m_ws10_test_proc_denorm()
test_ws10_denormed = test_t2m_ws10_denormed[0]
test = test_t2m_w10_denormed[1]

<IPython.core.display.Javascript object>

###  2. Data Split

In [15]:
# Train-test split into lead_times for t2m
X_train_t2m_denormed, y_train_t2m_denormed = split_var_lead_one(train_t2m_denormed)
X_test_t2m_denormed, y_test_t2m_denormed = split_var_lead_one(test_t2m_denormed)
# Train-test split into lead_times for ws10
X_train_ws10_denormed, y_train_ws10_denormed = split_var_lead_one(train_ws10_denormed)
X_test_ws10_denormed, y_test_ws10_denormed = split_var_lead_one(test_ws10_denormed)

<IPython.core.display.Javascript object>